In [50]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import models,layers
import numpy as np

In [2]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCH = 50

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "potato_dataset",
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    shuffle=True
 )

In [ ]:
classes_name  = dataset.class_names
classes_name

In [ ]:
#it shows 68 because 32 set of images are created as batches 2152 total make 67.25 batches 
len(dataset)

In [ ]:
plt.figure(figsize=(10,10))
for images, labels in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(4,3,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.axis("off")
        plt.title(classes_name[labels[i]])
    

In [7]:
train_size = 0.8
training_batch_size=int(len(dataset)*0.8)


In [ ]:
training_dataset = dataset.take(training_batch_size)
len(training_dataset)



In [ ]:
test_dataset = dataset.skip(54)
len(test_dataset)


In [ ]:
val_size = 0.1
val_dataset = test_dataset.take(6)
len(val_dataset)

In [ ]:
test_dataset = test_dataset.skip(6)
len(test_dataset)

In [12]:
def get_dataset_partitions_tf(ds,train_split=0.8,val_split=0.1,test_split=0.1,shuffle=True,shuffle_size=10000):
    
    if shuffle :
        ds = ds.shuffle(shuffle_size,seed=12)
        
    training_batch_size=int(len(ds)*train_split)
    val_batch_size=int(len(ds)*val_split)
        
    train_ds = dataset.take(training_batch_size)
    val_ds = dataset.skip(training_batch_size).take(val_batch_size)
    test_ds = dataset.skip(training_batch_size).skip(val_batch_size)
    return train_ds,val_ds,test_ds 

In [13]:
train_ds,val_ds,test_ds = get_dataset_partitions_tf(dataset)

In [14]:
train_ds = train_ds.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [15]:
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.Rescaling(1.0/255)
])

In [16]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])


In [ ]:
input_shape = (32,IMAGE_SIZE,IMAGE_SIZE,CHANNELS)
n_classes = 3

model = tf.keras.Sequential([
    resize_and_rescale,
    data_augmentation,
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(n_classes,activation='softmax')
])

model.build(input_shape=input_shape)

In [ ]:
model.summary()

In [19]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'],
    )


In [ ]:
history = model.fit(train_ds,epochs=50,batch_size = BATCH_SIZE,verbose=1,validation_data=val_ds)

In [ ]:
scores = model.evaluate(test_ds)

In [ ]:
for image_labels , batch_labels in test_ds.take(1):
    first_image =(image_labels[0].numpy().astype("uint8"))
    first_label = classes_name[batch_labels[0]]
    print(f"True Value :  {first_label}")
    plt.imshow(first_image)
    predictions = model.predict(image_labels)
    print(f'Predicted Value : {classes_name[np.argmax(predictions[0])]}')

In [68]:
def predict(model,img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array,0)

    predictions = model.predict(img_array)

    predicted_class = classes_name[np.argmax(predictions[0])]
    confidence = round(100*(np.max(predictions[0])),2)
    return predicted_class , confidence

In [ ]:
plt.figure(figsize = (12,12))
for images , labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        predicted_class , confidence  = predict(model,images[i].numpy())
        actual_class = classes_name[labels[i]]
        plt.title(f'Actual: {actual_class},\n Predicted: {predicted_class}')
        plt.axis('off')
        

In [ ]:
model_version = 1
model.save('Path to Save your model')
